<a href="https://colab.research.google.com/github/YannPhamVan/StockMarketsAnalyticsZoomcamp/blob/main/02-dataframe-analysis-homework/02-dataframe-analysis-homework-Q1-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module 2 Homework (2025 Cohort)

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---
#0) Imports and Installs

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

---
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**.

In [3]:
import pandas as pd
import requests
from io import StringIO

def get_withdrawn_ipos() -> pd.DataFrame:
    """
    Fetch withdrawn IPO data from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [4]:
withdrawn_ipos = get_withdrawn_ipos()
withdrawn_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [5]:
withdrawn_ipos

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-


2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Ltd” or “Limited” → `Limited`
   - “Holdings” → `Holdings`
   - Others → `Other`

  *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.


In [6]:
def get_company_class(company_name: str):
  ''' Apply a company name to get its class.
  '''
  cn = company_name.lower()
  if cn.find('acquisition corp')>=0 or cn.find('acquisition corporation')>=0:
    return 'Acq.Corp'
  elif cn.find('inc')>=0 or cn.find('incorporated')>=0:
    return 'Inc'
  elif cn.find('group')>=0:
    return 'Group'
  elif cn.find('ltd')>=0 or cn.find('limited')>=0:
    return 'Limited'
  elif cn.find('holdings')>=0:
    return 'Holdings'
  else:
    return 'Other'

withdrawn_ipos["Company Class"] = withdrawn_ipos["Company Name"].apply(get_company_class)

withdrawn_ipos

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc


In [7]:
withdrawn_ipos.value_counts('Company Class')

,count
Company Class,
Inc,51
Acq.Corp,21
Limited,17
Other,6
Group,4
Holdings,1


In [8]:
withdrawn_ipos.loc[withdrawn_ipos['Company Class'] == 'Other']

,Symbol,Company Name,Price Range,Shares Offered,Company Class
4,ONE,One Power Company,-,-,Other
9,KMCM,Key Mining Corp.,$2.25,4444444,Other
53,CLLB,"CoLabs Intâl, Corp.",$4.50,1300000,Other
74,TSIV,Twelve Seas Investment Company IV TMT,$10.00,20000000,Other
86,FSPR,Four Springs Capital Trust,$13.00 - $15.00,18000000,Other
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000,Other



3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`

In [9]:
def get_avg_price(price_range: str):
  '''
  Compute average price from price range.
  '''
  pr = price_range.replace('$', '')
  if pr == '-':
    return None
  elif pr.find('-') >= 0:
    return (float(pr.split('-')[0]) + float(pr.split('-')[1])) / 2
  else:
    return float(pr)

withdrawn_ipos["Avg. Price"] = withdrawn_ipos["Price Range"].apply(get_avg_price)

withdrawn_ipos

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN



4. Convert `Shares Offered` to numeric, clean missing or invalid values.

In [10]:
withdrawn_ipos["Shares Offered"] = pd.to_numeric(withdrawn_ipos["Shares Offered"], errors='coerce')
withdrawn_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          100 non-null    object 
 1   Company Name    100 non-null    object 
 2   Price Range     100 non-null    object 
 3   Shares Offered  72 non-null     float64
 4   Company Class   100 non-null    object 
 5   Avg. Price      73 non-null     float64
dtypes: float64(2), object(4)
memory usage: 4.8+ KB



5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)

In [11]:
withdrawn_ipos["Withdrawn Value"] = withdrawn_ipos["Shares Offered"] * withdrawn_ipos["Avg. Price"]
withdrawn_ipos.notna().sum()

,0
Symbol,100
Company Name,100
Price Range,100
Shares Offered,72
Company Class,100
Avg. Price,73
Withdrawn Value,71



6. Group by `Company Class` and calculate total withdrawn value.

In [12]:
withdrawn_ipos.groupby('Company Class').agg({'Withdrawn Value': 'sum'})

,Withdrawn Value
Company Class,
Acq.Corp,4.021000e+09
Group,3.378750e+07
Holdings,7.500000e+07
Inc,2.257164e+09
Limited,5.497346e+08
Other,7.679200e+08



7. **Answer**: Which class had the highest **total** value of withdrawals?
---

In [13]:
withdrawn_ipos.groupby('Company Class').agg({'Withdrawn Value': 'sum'}).sort_values('Withdrawn Value', ascending=False).iloc[0]

,Acq.Corp
Withdrawn Value,4.021000e+09


---
### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.


In [14]:
def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [15]:
ipos_2024 = get_ipos_by_year(2024)
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [16]:
ipos_2024["IPO Date"] = pd.to_datetime(ipos_2024["IPO Date"], format='mixed')

In [17]:
ipos_first_5_months_2024 = ipos_2024.loc[ipos_2024['IPO Date'] < '2024-06-01']


2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.

In [18]:
ALL_TICKERS = ipos_first_5_months_2024["Symbol"].to_list()
ALL_TICKERS

['NAKA',
 'BOW',
 'HDL',
 'RFAI',
 'JDZG',
 'RAY',
 'BTOC',
 'ZK',
 'GPAT',
 'PAL',
 'SVCO',
 'NNE',
 'CCIX',
 'VIK',
 'ZONE',
 'LOAR',
 'MRX',
 'RBRK',
 'NCI',
 'MFI',
 'YYGH',
 'TRSG',
 'CDTG',
 'CTRI',
 'IBTA',
 'MTEN',
 'SUPX',
 'TWG',
 'ULS',
 'PACS',
 'MNDR',
 'CTNM',
 'MAMO',
 'ZBAO',
 'BOLD',
 'MMA',
 'UBXG',
 'IBAC',
 'AUNA',
 'BKHA',
 'LOBO',
 'RDDT',
 'ALAB',
 'INTJ',
 'RYDE',
 'LGCL',
 'SMXT',
 'VHAI',
 'DYCQ',
 'CHRO',
 'UMAC',
 'HLXB',
 'MGX',
 'TBBB',
 'TELO',
 'KYTX',
 'PMNT',
 'AHR',
 'LEGT',
 'ANRO',
 'GUTS',
 'AS',
 'FBLG',
 'AVBP',
 'BTSG',
 'HAO',
 'CGON',
 'YIBO',
 'JL',
 'SUGP',
 'JVSA',
 'KSPI',
 'CCTG',
 'PSBD',
 'SYNX',
 'SDHC',
 'ROMA']

In [19]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90, 252, 365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 NAKA
1 BOW
2 HDL
3 RFAI
4 JDZG
5 RAY
6 BTOC
7 ZK
8 GPAT
9 PAL
10 SVCO
11 NNE
12 CCIX
13 VIK
14 ZONE
15 LOAR
16 MRX
17 RBRK
18 NCI
19 MFI
20 YYGH
21 TRSG
22 CDTG
23 CTRI
24 IBTA
25 MTEN
26 SUPX
27 TWG
28 ULS
29 PACS
30 MNDR
31 CTNM
32 MAMO
33 ZBAO
34 BOLD
35 MMA
36 UBXG
37 IBAC
38 AUNA
39 BKHA
40 LOBO
41 RDDT
42 ALAB
43 INTJ
44 RYDE
45 LGCL
46 SMXT
47 VHAI
48 DYCQ
49 CHRO
50 UMAC
51 HLXB
52 MGX
53 TBBB
54 TELO
55 KYTX
56 PMNT
57 AHR
58 LEGT
59 ANRO
60 GUTS
61 AS
62 FBLG
63 AVBP
64 BTSG
65 HAO
66 CGON
67 YIBO
68 JL
69 SUGP
70 JVSA
71 KSPI
72 CCTG
73 PSBD
74 SYNX
75 SDHC
76 ROMA


In [20]:
stocks_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_90d,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future
0,4.000,4.200,2.80,3.020,440600,0.0,0.0,NAKA,2024,5,...,NaN,NaN,NaN,0.784768,NaN,NaN,0,0.463576,NaN,0
1,2.990,3.110,2.35,2.660,147300,0.0,0.0,NAKA,2024,6,...,NaN,NaN,NaN,0.883459,NaN,NaN,0,0.285714,NaN,0
2,2.530,3.110,2.41,2.920,73800,0.0,0.0,NAKA,2024,6,...,NaN,NaN,NaN,0.955479,NaN,NaN,0,0.239726,NaN,0
3,2.910,3.090,2.60,2.730,51100,0.0,0.0,NAKA,2024,6,...,NaN,NaN,NaN,1.051282,NaN,NaN,0,0.179487,NaN,1
4,2.940,2.940,2.41,2.690,56500,0.0,0.0,NAKA,2024,6,...,NaN,NaN,NaN,1.111524,NaN,NaN,0,0.197026,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23798,3.021,3.050,2.85,2.880,36300,0.0,0.0,ROMA,2025,6,...,4.241532,5.938144,NaN,NaN,3.2390,2.78955,1,0.069444,10.886753,0
23799,2.870,2.890,2.56,2.660,123100,0.0,0.0,ROMA,2025,6,...,3.917526,4.666667,NaN,NaN,3.1970,2.82155,1,0.124060,10.383358,0
23800,2.840,3.000,2.64,2.875,63100,0.0,0.0,ROMA,2025,6,...,4.342900,5.424529,NaN,NaN,3.1485,2.86180,1,0.125217,9.939668,0
23801,2.850,2.935,2.79,2.795,10200,0.0,0.0,ROMA,2025,6,...,4.092240,5.008960,NaN,NaN,3.0870,2.89955,1,0.051878,9.469241,0





3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
   ```

   ⚠️ **IMPORTANT** Please use the original version of annualized volatility calculation (it was later corrected to another formula):
   ```python
   stocks_df['volatility'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
   ```

In [21]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']


4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

In [22]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], format='mixed')

In [23]:
stocks_df.loc[stocks_df['Date'] == '2025-06-06'].describe()[["growth_252d", "Sharpe"]]

,growth_252d,Sharpe
count,73.000000,73.000000
mean,1.227948,0.284576
min,0.024970,-0.079677
25%,0.293510,0.040265
50%,0.763188,0.083768
75%,1.446667,0.291048
max,8.097413,2.835668
std,1.480237,0.512601




5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  **0.083768**
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`? **Not really as only one Ticker is common to the 2 lists**

---

In [24]:
stocks_df.loc[stocks_df['Date'] == '2025-06-06'].sort_values(by="growth_252d", ascending=False).head(10)["Ticker"]

,Ticker
20976,JL
23794,ROMA
254,NAKA
14800,UMAC
3211,NNE
4895,RBRK
17183,AHR
18526,AS
7521,SUPX
4607,MRX


In [25]:
stocks_df.loc[stocks_df['Date'] == '2025-06-06'].sort_values(by="Sharpe", ascending=False).head(10)["Ticker"]

,Ticker
11413,BKHA
21648,JVSA
17490,LEGT
10826,IBAC
15140,HLXB
8709,MNDR
14128,DYCQ
12662,INTJ
20976,JL
6059,TRSG


---
### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

In [53]:
def compute_future_growth(df, months=12):
    df_sorted = df.sort_values(['Ticker', 'Date'])
    df_sorted = df_sorted.set_index('Date')

    for m in range(1, months+1):
        df_sorted[f'growth_future_{m}m'] = (
            df_sorted.groupby('Ticker')['Close'].shift(-21 * m) / df_sorted['Close']
        )
    return df_sorted.reset_index()

future_growth_df = compute_future_growth(stocks_df)
future_growth_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
0,2024-02-07,12.085785,12.471402,11.878869,12.433781,12732800,0.0,0.0,AHR,2024,...,1.060660,1.125315,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
1,2024-02-08,12.245675,12.452591,12.226865,12.283297,1630300,0.0,0.0,AHR,2024,...,1.094691,1.156243,1.267090,1.324937,1.817033,1.951132,2.105590,2.256846,2.197812,2.230109
2,2024-02-09,12.236269,12.650101,12.179837,12.339726,2062100,0.0,0.0,AHR,2024,...,1.086582,1.157160,1.262084,1.352007,1.803991,1.927870,2.077638,2.219440,2.180527,2.207051
3,2024-02-12,13.120365,13.694088,12.320917,12.367943,1161200,0.0,0.0,AHR,2024,...,1.090294,1.151425,1.278880,1.390634,1.864410,1.953675,2.103896,2.231068,2.194798,2.278196
4,2024-02-13,12.367942,12.753559,12.236269,12.556048,1631700,0.0,0.0,AHR,2024,...,1.067100,1.131888,1.289178,1.399259,1.905473,1.933801,2.028534,2.218782,2.205361,2.222739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23798,2025-06-12,3.800000,3.800000,3.510000,3.590000,179500,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23799,2025-06-13,3.710000,3.710000,3.170000,3.290000,79800,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23800,2025-06-16,3.120000,3.290000,2.839000,3.080000,97000,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23801,2025-06-17,2.990000,3.240000,2.915000,3.160000,32700,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

In [54]:
min_date_df = future_growth_df.groupby('Ticker').Date.agg('min').reset_index()
min_date_df

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
...,...,...
72,YIBO,2024-01-25
73,YYGH,2024-04-22
74,ZBAO,2024-04-02
75,ZK,2024-05-10






3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

In [55]:
joined_df = future_growth_df.merge(min_date_df, on=['Ticker', 'Date'], how='inner')
joined_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
0,2024-02-07,12.085785,12.471402,11.878869,12.433781,12732800,0.0,0.0,AHR,2024,...,1.060660,1.125315,1.214961,1.308118,1.753552,1.963886,1.988395,2.257203,2.259751,2.273312
1,2024-03-20,52.560001,63.500000,50.610001,62.029999,16843300,0.0,0.0,ALAB,2024,...,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,2024-02-02,22.000000,23.270000,20.000000,20.700001,2386300,0.0,0.0,ANRO,2024,...,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,2024-02-01,13.400000,13.800000,13.100000,13.400000,18656400,0.0,0.0,AS,2024,...,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,2024-03-22,9.510000,10.320000,9.300000,9.600000,9046900,0.0,0.0,AUNA,2024,...,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2024-01-25,4.000000,4.000000,2.400000,2.790000,875500,0.0,0.0,YIBO,2024,...,0.734767,0.788530,0.723656,1.044803,0.777778,0.957706,0.996416,1.290323,1.326165,1.842294
73,2024-04-22,3.880000,4.700000,2.580000,2.900000,990300,0.0,0.0,YYGH,2024,...,0.246897,0.255172,0.315517,0.431034,0.631034,0.675862,0.634483,0.606897,0.617241,0.355172
74,2024-04-02,4.000000,4.000000,3.000000,3.700000,755000,0.0,0.0,ZBAO,2024,...,1.075676,1.037838,0.891892,1.005405,0.851351,0.770270,0.424324,0.418919,0.370270,0.300541
75,2024-05-10,26.000000,29.360001,25.510000,28.260000,9709800,0.0,0.0,ZK,2024,...,0.508139,0.598726,0.869073,0.878627,0.998231,0.961076,0.877212,1.105096,0.756546,1.018401




4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

In [58]:
joined_df.describe()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,Month,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
count,77,77.000000,77.000000,77.000000,77.000000,7.700000e+01,77.0,77.0,77.0,77.000000,...,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,76.000000,76.000000,73.000000
mean,2024-03-25 12:09:21.038961152,15.307958,17.477002,14.014424,15.528368,5.237076e+06,0.0,0.0,2024.0,3.311688,...,0.834120,0.822711,0.802881,0.864347,0.846270,0.829974,0.878871,0.913272,0.896660,0.991712
min,2024-01-09 00:00:00,0.010700,0.015700,0.006200,0.011600,6.000000e+02,0.0,0.0,2024.0,1.000000,...,0.060947,0.045368,0.054109,0.061432,0.048274,0.043103,0.033144,0.041357,0.023674,0.038947
25%,2024-02-09 00:00:00,4.430000,5.400000,3.595000,4.070000,7.114000e+05,0.0,0.0,2024.0,2.000000,...,0.508139,0.515385,0.445378,0.387417,0.324100,0.225974,0.232468,0.248485,0.268982,0.229851
50%,2024-03-28 00:00:00,10.000000,10.110000,9.980000,10.030000,1.992600e+06,0.0,0.0,2024.0,3.000000,...,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.738029,0.710647
75%,2024-04-25 00:00:00,21.750000,23.270000,19.049999,19.799999,5.181000e+06,0.0,0.0,2024.0,4.000000,...,1.075676,1.125315,1.017822,1.103896,1.150909,1.074739,1.037624,1.194942,1.169029,1.189424
max,2024-09-11 00:00:00,103.250000,103.500000,95.029999,98.000000,4.870550e+07,0.0,0.0,2024.0,9.000000,...,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,5.672185
std,NaN,17.417806,19.150602,16.570734,18.105833,8.789692e+06,0.0,0.0,0.0,1.656405,...,0.410362,0.401473,0.486716,0.649114,0.706456,0.754255,0.925134,0.900345,0.859311,1.061934




5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).

**Answer = 12**

In [59]:
joined_df[joined_df.columns[-12:]].describe().iloc[1].sort_values(ascending=False)

,mean
growth_future_12m,0.991712
growth_future_2m,0.936866
growth_future_1m,0.926390
growth_future_10m,0.913272
growth_future_11m,0.896660
growth_future_9m,0.878871
growth_future_6m,0.864347
growth_future_7m,0.846270
growth_future_3m,0.834120
growth_future_8m,0.829974
